# Assignment 3 - Building a Custom Visualization

---

In this assignment you must choose one of the options presented below and submit a visual as well as your source code for peer grading. The details of how you solve the assignment are up to you, although your assignment must use matplotlib so that your peers can evaluate your work. The options differ in challenge level, but there are no grades associated with the challenge level you chose. However, your peers will be asked to ensure you at least met a minimum quality for a given technique in order to pass. Implement the technique fully (or exceed it!) and you should be able to earn full grades for the assignment.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ferreira, N., Fisher, D., & Konig, A. C. (2014, April). [Sample-oriented task-driven visualizations: allowing users to make better, more confident decisions.](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;In Proceedings of the SIGCHI Conference on Human Factors in Computing Systems (pp. 571-580). ACM. ([video](https://www.youtube.com/watch?v=BI7GAs-va-Q))


In this [paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) the authors describe the challenges users face when trying to make judgements about probabilistic data generated through samples. As an example, they look at a bar chart of four years of data (replicated below in Figure 1). Each year has a y-axis value, which is derived from a sample of a larger dataset. For instance, the first value might be the number votes in a given district or riding for 1992, with the average being around 33,000. On top of this is plotted the 95% confidence interval for the mean (see the boxplot lectures for more information, and the yerr parameter of barcharts).

<br>
<img src="readonly/Assignment3Fig1.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Figure 1 from (Ferreira et al, 2014).</h4>

<br>

A challenge that users face is that, for a given y-axis value (e.g. 42,000), it is difficult to know which x-axis values are most likely to be representative, because the confidence levels overlap and their distributions are different (the lengths of the confidence interval bars are unequal). One of the solutions the authors propose for this problem (Figure 2c) is to allow users to indicate the y-axis value of interest (e.g. 42,000) and then draw a horizontal line and color bars based on this value. So bars might be colored red if they are definitely above this value (given the confidence interval), blue if they are definitely below this value, or white if they contain this value.


<br>
<img src="readonly/Assignment3Fig2c.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  Figure 2c from (Ferreira et al. 2014). Note that the colorbar legend at the bottom as well as the arrows are not required in the assignment descriptions below.</h4>

<br>
<br>

**Easiest option:** Implement the bar coloring as described above - a color scale with only three colors, (e.g. blue, white, and red). Assume the user provides the y axis value of interest as a parameter or variable.


**Harder option:** Implement the bar coloring as described in the paper, where the color of the bar is actually based on the amount of data covered (e.g. a gradient ranging from dark blue for the distribution being certainly below this y-axis, to white if the value is certainly contained, to dark red if the value is certainly not contained as the distribution is above the axis).

**Even Harder option:** Add interactivity to the above, which allows the user to click on the y axis to set the value of interest. The bar colors should change with respect to what value the user has selected.

**Hardest option:** Allow the user to interactively set a range of y values they are interested in, and recolor based on this (e.g. a y-axis band, see the paper for more details).

---

*Note: The data given for this assignment is not the same as the data used in the article and as a result the visualizations may look a little different.*

In [ ]:
# Use the following data for this assignment:

import pandas as pd
import numpy as np

np.random.seed(12345)

df = pd.DataFrame([np.random.normal(32000,200000,3650), 
                   np.random.normal(43000,100000,3650), 
                   np.random.normal(43500,140000,3650), 
                   np.random.normal(48000,70000,3650)], 
                  index=[1992,1993,1994,1995])
df

## Initialize imports, logger

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation, gridspec, widgets, cm
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import logging, sys
import scipy.stats as st

log = logging.getLogger('assigment3')
logging.basicConfig()


In [ ]:
%matplotlib notebook

## Calculate statistics on data

### Mean, standard error, and degrees of freedom

In [ ]:
dfT = df.sort_index().T
stats = pd.DataFrame({'mean': dfT.apply(np.mean),
                      'stderr': dfT.apply(st.sem),
                      'dof': dfT.count()-1})

### 95% confidence interval

In [ ]:
ar1, ar2 = st.t.interval(0.95, stats['dof'], loc=stats['mean'], scale=stats['stderr'])
stats['interval'] = stats['mean'] - ar1
stats

### 1-sample t-test, for calculating chance the mean is different than the hypothesis value, given the sample

In [ ]:
res = st.ttest_1samp(dfT, 40000)

### Cumulative density function, for finding the portion of mean probability between two values

In [ ]:
st.t.cdf(40000, stats['dof'], loc=stats['mean'], scale=stats['stderr'])

## Helper methods for color map

In [ ]:
def p_to_norm(p, higher):
    """Scale a p value to a value between 0 and 1 for the color map. choose a scale to adjust the rate of gradient change"""
    scaled = np.power(p, 0.3)
    return (0.5 + (1-scaled)/2) if higher else scaled/2

def p_to_color_old(p, higher=False):
    """Convert of p value to a color hex code in form '#rrggbb'.
higher=True if the observed mean (bar height) is higher than the test mean."""
#     get_hex = lambda fl: hex(int(np.floor(p*256)))[2:4]
    get_hex = lambda fl: int(np.floor(p*256))
    g = get_hex(p)
    r = 255 if higher else get_hex(p)
    b = 255 if not higher else get_hex(p)
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)

def p_to_color_div(p, higher=False):
    """Convert of p value to a color code using a diverging
color map, meaning two opposite poles are distinguished from the middle and from each other.
higher=True if the observed mean (bar height) is higher than the test mean."""
    cmap = cm.get_cmap('seismic')
#     help(cmap)
    return cmap(p_to_norm(p, higher))


def p_to_color_seq(p):
    """Convert of p value to a color code using a sequential
color map, meaning the distinction is made primarily between two opposite poles."""
    cmap = cm.get_cmap('Reds')
#     help(cmap)
    return cmap(p)

# p_to_color(0.0001, False), p_to_color(0.3, True)
plt.figure()
# Plot map from p to geometric curve
plt.plot(np.arange(0, 1, 0.05), [p_to_norm(p, False) for p in np.arange(0, 1, 0.05)])

## Create an interactive plot that supports both 1-sample tests against a chosen value and a test for how likely the population mean is to be within a range

In [ ]:
gs = gridspec.GridSpec(2, 1, height_ratios=(6,1), hspace=0.3)
fig = plt.figure(figsize=(9,7))
ax = plt.subplot(gs[0, 0])
ax2 = plt.subplot(gs[1, 0])

y1, y2 = 40000, 40000
cursor, lines = None, []


def redraw():
    """Redraw the basic subplot and bars"""
    global cursor
    cursor = None
    ax.cla()
    
    bars = ax.bar(stats.index.astype(str), stats['mean'], yerr=stats['interval'], capsize=10);
    fig.subplots_adjust(right=.75)
    
    ax.set_frame_on(False)
    ax.tick_params(bottom=False)
    return bars

def plot(stats):
    """Plot the bars, draw all supporting features, run tests, paint according to test results."""
    global y1, y2, lines
    bars = redraw()
    
    if y1 == y2:
        add_line(y1)
        ax.set_title('Mean comparison against y = {}'.format(int(y1)))

        ttres = st.ttest_1samp(dfT, y1)
        ps = ttres[1]
        
        label_bars(ps, bars, lambda p,b: p_to_color_div(p, b.get_height() > y1), True)

        asc, desc = np.arange(0,1,0.2), np.arange(1,-0.1,-0.2)
        colors = [p_to_color_div(p, True) for p in asc] + [p_to_color_div(p, False) for p in desc]

        leg = add_legend(colors, np.around(np.append(asc, desc), 1))
    else:
        add_line(y1)
        add_line(y2)
        ymin, ymax = min(y1, y2), max(y1, y2)
        
        ax.set_title('Probability of population mean between {} and {}'.format(int(ymin), int(ymax)))
        
        lower = st.t.cdf(ymin, stats['dof'], loc=stats['mean'], scale=stats['stderr'])
        higher = st.t.cdf(ymax, stats['dof'], loc=stats['mean'], scale=stats['stderr'])
        density_in_range = higher - lower
        
        label_bars(density_in_range, bars, lambda p,b: p_to_color_seq(p), False)

        seq = np.arange(1.01,0,-0.1)
        colors = [p_to_color_seq(p) for p in seq]

        leg = add_legend(colors, np.around(seq, 1))
    
    return bars

def label_bars(ps, bars, ptc, invert_colors=False):
    for p,b in zip(ps, bars):
        textcolor = 'black' if invert_colors ^ (p < 0.40) else 'white'
        ax.text(b.get_x() + b.get_width()/2, 8000, '\u03bc: {}'.format(int(b.get_height())),
                  ha='center', va='center', color=textcolor)
        ax.text(b.get_x() + b.get_width()/2, 5000, 'p: {:.2}'.format(p),
                  ha='center', va='center', color=textcolor)
        b.set_color(ptc(p, b))

def setup_widgets():
    ax2.set_frame_on(False)
    ax2.tick_params(bottom=False, labelbottom=False, left=False, labelleft=False)
    ax2.text(-0.1,1, ("Directions: Click once to establish a single mean value to compare against "
                   "depicted means.\nClick and drag to establish a vertical range to assess how "
                   "likely each mean is to fall within the range."))
    
def add_line(y):
    global lines
    lines.append(ax.add_line(Line2D([-0.5, 3.5], [y,y], linewidth=0.5, color='black')))
    
def clear_lines():
    global lines
    for l in lines: l.set_visible(False)
    lines.clear()
    
def add_legend(colors, labels):
    leg = ax.legend(handles=[Patch(facecolor=c, label=l) for c, l in
                             zip(colors, labels)
                            ],
                    loc=(0.98,0.1), title='p-value')
    leg.set_frame_on(False)
    return leg

def onpress(event):
    global y1, cursor, lines
    y1 = event.ydata
    clear_lines()
    add_line(y1)
    cursor = widgets.Cursor(ax, vertOn=False, color='black', linestyle='--')
    
def onrelease(event):
    global y2
    y2 = event.ydata
    plot(stats)
    
    
# tell mpl_connect we want to pass a 'button_press_event' into onclick when the event is detected
fig.canvas.mpl_connect('button_press_event', onpress)
fig.canvas.mpl_connect('button_release_event', onrelease)

plot(stats)
setup_widgets()